In [1]:
!pip install html5lib
!pip install tqdm
!pip install pandas
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import html5lib


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
url = "https://us.soccerway.com/international/world/world-cup/2022-qatar/group-stage/r49519/"
group_div = "block_competition_tables_group-wrapper"

my_agent = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"}
response = requests.get(url, headers = my_agent)
soup = BeautifulSoup(response.text)

all_current = soup.find_all("div", {"class": group_div})

team_list = []

for div in all_current:
    
    tables = div.findChildren("table", recursive=True)
    table = tables[0]
    
    trs = table.findChildren("tr", recursive=True)

    for tr in trs:
        tds = tr.findChildren("td", recursive=True)
        for td in tds:
            classes = td.get("class")
            if 'large-link' in classes:
                team_name = td.findChildren("a")[0].contents[0]
                href = td.findChildren("a")[0]["href"]
                
                base_url = "https://us.soccerway.com/"
                link = f'{base_url}/{href}'
                
                team = {
                    "name": team_name,
                    "link": link,
                }
                
                team_list.append(team)
            
data_frame = pd.DataFrame(data=team_list)
print(data_frame)

              name                                               link
0      Netherlands  https://us.soccerway.com///teams/netherlands/n...
1          Ecuador  https://us.soccerway.com///teams/ecuador/ecuad...
2          Senegal  https://us.soccerway.com///teams/senegal/seneg...
3            Qatar  https://us.soccerway.com///teams/qatar/qatar/1...
4          England  https://us.soccerway.com///teams/england/engla...
5             Iran   https://us.soccerway.com///teams/iran/iran/1178/
6    United States  https://us.soccerway.com///teams/united-states...
7            Wales  https://us.soccerway.com///teams/wales/wales/2...
8     Saudi Arabia  https://us.soccerway.com///teams/saudi-arabia/...
9           Poland  https://us.soccerway.com///teams/poland/poland...
10          Mexico  https://us.soccerway.com///teams/mexico/mexico...
11       Argentina  https://us.soccerway.com///teams/argentina/arg...
12          France  https://us.soccerway.com///teams/france/france...
13         Tunisia  

In [26]:
point_list = []

for team in team_list:
    match_url = f'{team["link"]}matches/'

    print(match_url)

    response = requests.get(match_url, headers = my_agent)
    soup = BeautifulSoup(response.text)

    tables = soup.findAll("table", recursive=True)
    table = tables[0]

    tbody = table.findChildren('tbody')[0]
    trs = tbody.findChildren('tr');

    index = 0;

    for tr in trs:
        tds = tr.findChildren('td')
        match_type_cell = tds[1]
        _as = match_type_cell.findChildren('a')
        _a = _as[0]
        match_type = _a.contents[0]

        if match_type == "WOC":
            break

        index += 1

    amount = 5;
    index -= 1;
    indecies = range(index, index - amount, -1)

    points = 0
    for i in indecies:
        tr = trs[i]
        tds = tr.findChildren('td');
        result = tds[3]

        _as = result.findChildren('a')
        a = _as[0]

        result_classes = a.get("class")
        print(result_classes)
        
        # cancelled games wont have any classes.
        # we skip the point calculation in that
        # case, because the next line would throw
        # an "index out of range" error if we would
        # run it.
        if not result_classes:
            continue
        
        result_class = result_classes[0]

        if result_class == "result-win":
            points += 3
        elif result_class == "result-draw":
            points += 1

    point_list.append(points);
        
data_frame['points'] = point_list

print(data_frame)

https://us.soccerway.com///teams/netherlands/netherlands/1552/matches/
['result-win']
['result-win']
['result-win']
['result-draw']
['result-win']
https://us.soccerway.com///teams/ecuador/ecuador/657/matches/
['result-draw']
['result-draw']
['result-draw']
['result-win']
['result-draw']
https://us.soccerway.com///teams/senegal/senegal/1941/matches/
['result-draw']
['result-win']
['result-win']
['result-win']
['result-loss']
https://us.soccerway.com///teams/qatar/qatar/1774/matches/
['result-win']
['result-win']
['result-win']
['result-win']
['result-draw']
https://us.soccerway.com///teams/england/england/774/matches/
['result-draw']
['result-loss']
['result-loss']
['result-draw']
['result-draw']
https://us.soccerway.com///teams/iran/iran/1178/matches/
['result-win']
['result-loss']
[]
['result-win']
['result-loss']
https://us.soccerway.com///teams/united-states/united-states-of-america/2281/matches/
['result-draw']
['result-loss']
['result-draw']
['result-win']
['result-draw']
https://